In [0]:
CREATE OR REPLACE VIEW workspace.gold.vw_investor_relation AS
  SELECT 
  investor.investor_id,
  investor.investor_type,
  transact.transaction_type,
  transact.fund,
  investment.investment,
  investment.entry_date,
  investment.exit_date,
  transact.gl_date,
  transact.amount,
  transact.export_date
  FROM workspace.gold.fact_transaction AS transact  
  LEFT JOIN workspace.gold.dim_investor AS investor
  ON transact.investor_id = investor.investor_id
  LEFT JOIN workspace.gold.dim_investment AS investment
  ON transact.investment = investment.investment


# Defining Problem Statments

##### 1. Summarize overall performance of fund 'Fund Two' for Investor 7002.
##### 2. Compare our returns vs. other Fund Two investors.
##### 3. Show trend of MoM since initial investments.
##### 4. Identify post-hoc adjustments. 

## 1 - Summarize overall performance of fund 'Fund Two' for Investor 7002

In [0]:
SELECT
  investor_id,
  fund,  
  ROUND(SUM(CASE WHEN LOWER(transaction_type) in ('expense', 'contribution') THEN amount ELSE 0 END), 2) AS total_invested,
  ROUND(SUM(CASE WHEN LOWER(transaction_type) = 'distribution' THEN amount ELSE 0 END), 2) AS total_returned,
  ROUND(total_returned / NULLIF(total_invested,0),2) as multiple_on_money
FROM workspace.gold.vw_investor_relation
WHERE 1=1
AND LOWER(fund) = 'fund_two'
and investor_id = 7002
GROUP BY investor_id, fund
ORDER BY investor_id;

## 2 - Compare Investor 7002 vs. other Fund Two Investors

In [0]:
WITH investor_mom AS (
  SELECT 
  investor_id,
  ROUND(
    SUM(CASE WHEN LOWER(transaction_type) in ('expense', 'contribution') THEN amount ELSE 0 END) / 
    NULLIF(SUM(CASE WHEN LOWER(transaction_type) = 'distribution' THEN amount ELSE 0 END), 0), 2
   ) AS MOM
  FROM workspace.gold.vw_investor_relation
  WHERE 1=1
  AND LOWER(fund) = 'fund_two'
  GROUP BY investor_id
)
SELECT
  a.investor_id,
  a.mom AS investor_7002_mom,  
  ROUND(AVG(b.mom), 2) AS avg_mom_all_investors,
  ROUND(a.mom - AVG(b.mom), 2) AS diff_from_avg
FROM investor_mom AS a
CROSS JOIN investor_mom AS b
WHERE 1=1
AND a.investor_id = 7002
GROUP BY a.investor_id, a.mom
ORDER BY a.investor_id

## 3 - Show trend of MoM since initial investments

In [0]:
-- WITH txn AS (
WITH txn AS (
  SELECT
    investor_id,
    fund,
    DATE_TRUNC('quarter', export_date) AS quarter_start,
    SUM(CASE WHEN LOWER(transaction_type) in ('expense', 'contribution') THEN amount ELSE 0 END) AS total_invested,
    SUM(CASE WHEN LOWER(transaction_type) = 'distribution' THEN amount ELSE 0 END) AS total_returned
  FROM workspace.gold.vw_investor_relation
  WHERE 1=1
  AND LOWER(fund) = 'fund_two'
  AND investor_id = 7002
  GROUP BY investor_id, fund, DATE_TRUNC('quarter', export_date)
),
cumulative AS (
  SELECT 
    investor_id,
    fund,
    quarter_start,
    SUM(total_invested) OVER(PARTITION BY investor_id, fund ORDER BY quarter_start ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_inflow,
    SUM(total_returned) OVER(PARTITION BY investor_id, fund ORDER BY quarter_start ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS cum_outflow
  FROM txn
)
SELECT
  investor_id,
  fund,
  CAST(quarter_start AS DATE) AS quarter_start,
  ROUND(cum_outflow, 2) AS cum_outflow,
  ROUND(cum_inflow, 2) AS cum_inflow,
  ROUND(cum_outflow / NULLIF(cum_inflow,0),2) AS cumulative_mom
FROM cumulative
ORDER BY quarter_start;

## 4 - Identify post-hoc adjustments

In [0]:
WITH last_quarter AS (
  SELECT MAX(export_date) AS last_quarter
  FROM workspace.gold.vw_investor_relation
  WHERE export_date < (SELECT MAX(export_date) FROM workspace.gold.vw_investor_relation)
)
SELECT
  investor_id,
  fund,
  transaction_type,
  investment,
  amount,
  export_date
FROM workspace.gold.vw_investor_relation
WHERE 1=1
AND LOWER(fund) = 'fund_two'
AND investor_id = 7002
AND export_date > (SELECT last_quarter FROM last_quarter)
ORDER BY investment ASC

In [0]:


FROM workspace.gold.vw_investor_relation
WHERE 1=1
AND LOWER(fund) = 'fund_two'
AND investor_id = 7002
AND export_date >= '202
GROUP BY investor_id, fund